# Supplemental figures

This notebook is used to generate rasterplots of responses to stimulus sequences. This doesn't work without neurobank.

In [ ]:
import sys

sys.path.insert(0, "../scripts")

In [ ]:
import json
import string
from pathlib import Path

import graphics_defaults  # noqa: F401
import matplotlib.pyplot as plt
import numpy as np
from core import (
    NullSplitter,
    load_wave,
    split_trials,
)
from dlab import nbank, plotting
from filters import SpectrogramTransform
from matplotlib.patches import Rectangle

In [ ]:
dataset_dir = Path("../datasets/zebf-social-acoustical-ephys")
metadata_dir = dataset_dir / "metadata/"
response_dir = dataset_dir / "responses/"
stim_dir = dataset_dir / "stimuli"
# spectrogram parameters
window_size = 0.020
max_frequency = 8000

In [ ]:
# Example 1: not very selective, very discriminable, modest noise invariance
unit_name = "C42_4_1_c14"
seq_select = [0, 1, 2]

In [ ]:
# Example 2? selective and noise-invariant, but distance matrix is messy
unit_name = "C42_4_1_c131"

In [ ]:
# Example 2?
unit_name = "C42_1_1_c294"

In [ ]:
# Example 2?
unit_name = "C104_3_1_c67"
seq_select = [0, 1, 3]

In [ ]:
pprox_file = nbank.find_resource(unit_name, alt_base=response_dir)
unit = json.loads(pprox_file.read_text())
trials = split_trials(NullSplitter(), unit, metadata_dir).swaplevel().sort_index(ascending=False)

## Basic plot

In [ ]:
colors = {
    v: c for v, c in zip(trials.index.unique(level=0), plt.color_sequences["tab10"])
}
fig, ax = plt.subplots(nrows=1, figsize=(8,4))
for i, trial in enumerate(trials.itertuples()):
    if isinstance(trial.events, float):
        continue
    stim = trial.Index[0]
    ax.plot(trial.events,
                 [i] * trial.events.size,
                 color=colors[stim],
                 marker="|",
                 linestyle="",)
plotting.adjust_raster_ticks(ax, gap=1)
plotting.simple_axes(ax)

## Fancy plot

The idea here is to show a schematic view of the stimulus above the raster.

In [ ]:
sequence_names = trials.index.unique(level=0)
motif_labels = {motif: string.ascii_lowercase[i] for i, motif in enumerate(sequence_names[0].split("-"))}

In [ ]:
# the foreground sequences themselves are not in the registry, so to get the motif boundaries we have to look up one real stimulus per foreground
sequence_exemplar_names = tuple(trial["stimulus"]["name"] for trial in unit["pprox"] if trial["stimulus"]["name"].endswith("-100"))
sequence_info = {info["metadata"]["foreground"]: info["metadata"] for info in nbank.describe_many(nbank.default_registry, *sequence_exemplar_names)}

In [ ]:
def plot_stimulus_cartoon(ax, seq):
    seq_info = sequence_info[seq]
    duration = seq_info["background-duration"]
    motifs = seq.split("-")
    ax.add_patch(Rectangle(xy=(0,0), width=duration, height=1.0, facecolor="slateblue"))
    for motif, begin, end in zip(motifs, seq_info["stim_begin"], seq_info["stim_end"]):
        label = motif_labels[motif]
        ax.add_patch(Rectangle(xy=(begin, 0.0), width=end-begin, height=1.0, facecolor="tomato"))
        ax.text((end + begin) / 2, 0.5, label, fontsize=10, color="white", ha="center", va="center")
    ax.set_xlim(0, duration)
    plotting.hide_axes(ax)

In [ ]:
def plot_trials(ax, trials):
    # from matplotlib.ticker import FuncFormatter
    # background_values = trials.index.get_level_values("background-dBFS")
    # formatter = FuncFormatter(lambda x, pos: background_values[int(x)])
    for i, trial in enumerate(trials.itertuples()):
        if isinstance(trial.events, float):
            continue
        _stim = trial.Index[0]
        ax.plot(trial.events,
                [i] * trial.events.size,
                color="k",
                marker="|",
                linestyle="",
                markeredgewidth=0.4
               )
    plotting.simple_axes(ax)
    ax.get_yaxis().set_visible(False)
    ax.spines["left"].set_visible(False)

In [ ]:
seq = sequence_names[2]
fig, ax = plt.subplots(nrows=2, sharex=True, figsize=(4.0, 0.75), height_ratios=(2,4))
fig.subplots_adjust(left=0.01, right=0.99, hspace=0)
plot_stimulus_cartoon(ax[0], seq)
plot_trials(ax[1], trials.xs(seq, drop_level=False))
plotting.adjust_raster_ticks(ax[1], 1.5)

In [ ]:
nseq = len(seq_select)
# annoying hack to get stimulus name without string formatting
seq0_name = sequence_names[seq_select[0]]
seq0_trial = trials.loc[seq0_name, -100].source_trial
stim0_name = unit["pprox"][seq0_trial]["stimulus"]["name"]
stim0_info = nbank.describe(nbank.default_registry, stim0_name)["metadata"]
background_file = nbank.find_resource(stim0_info["background"], alt_base=stim_dir)
scene_file = nbank.find_resource(stim0_name, alt_base=stim_dir)

In [ ]:
fig, axs = plt.subplots(nrows=2 * nseq + 2, sharex=True, figsize=(3.0, 0.7 * (nseq + 1)), height_ratios=(2,2) + (2,4) * nseq, dpi=300)
fig.subplots_adjust(left=0.01, right=0.99, hspace=0.05)
for ax, fname in zip(axs, (background_file, scene_file)):
    signal = load_wave(fname)
    stfter = SpectrogramTransform(window_size, signal["sampling_rate"], max_frequency)
    spec = stfter.transform(signal["signal"], scaling=None) + 1e-6
    log_spec = 10 * np.log10(spec / stfter.scale1)
    fgrid = stfter.freq
    tgrid = stfter.tgrid(spec)
    pos = ax.imshow(log_spec, aspect="auto", origin="lower", vmin=-90, vmax=-20, 
                   extent=(tgrid[0], tgrid[-1], fgrid[0] / 1000, fgrid[-1] / 1000))
    ax.set_yticks([1, 8])
    ax.set_ylim(0.2, 8)
for i, seq in zip(range(2, nseq * 2 + 2, 2), seq_select):
    seq_name = sequence_names[seq]
    plot_stimulus_cartoon(axs[i], seq_name)
    plot_trials(axs[i+1], trials.xs(seq_name, drop_level=False))
    plotting.adjust_raster_ticks(axs[i+1], 8.5)
for ax in axs[:-2]:
    ax.get_xaxis().set_visible(False)
    ax.spines["bottom"].set_visible(False)
axs[-1].set_xlabel("Time (s)");

In [ ]:
fig.savefig(f"../figures/{unit_name}_sequence_rasters.pdf")